When working with data we often do not know the entire data set in advance. For example, think of the dataset of all Google queries, this dataset is constantly changing and becoming larger. This is what leads us to the idea of data streams. These data streams are potentially infinite and non-stationary i.e their distributions are constantly changing.

## Data Stream Model

Let us take a closer look at what data streams look like. We have a system that is receiving input at a rapid rate from one or more input ports (each port being a separate stream). We also think of this input as elements or tuples of data.

Now we want to make decisions or calculations using these data streams. But we run into a few problems, the data is constantly changing and we can not fit all the data in memory, be it secondary or primary. So we need to develop some algorithms to either sample the data or make approximations. 

:::example

Think of Google as the system. They have multiple data streams, one could search queries another maybe from Gmail or Google Drive etc. If we take a look at the search query stream maybe google wants to find out which search queries are popular at the minute to create a trending page.

:::

![dataStreamModel](/img/programming/dataStreamModel.png)

## Sampling a Data Stream

Since we can't store and use the entire data stream we want to take samples of a data stream. However, we want these samples to be fair and represent the entire data stream which can make things very complicated, for example, if we want a sample size of 100 elements we can not just keep track of the last 100 elements as this is not a fair representation of the entire data stream.

When working with samples we are interested in two types of samples either a fixed proportion of the data stream for example 10% of all elements. Or we can get a random sample of fixed size for example 100 elements.

![dataSample](/img/programming/dataSample.png)

### Fixed Proportion Sample

When working with a fixed proportion sample we, for example, want 1 in 10 elements, i.e 10% of the data. However, this does have some issues, if the data stream is not very large then 10% might not be enough data to do our task, be it a calculation, decision or prediction etc. 

You can also imagine the opposite, since the data stream is infinitely large this fixed proportion will grow infinitely which could lead to use not being able to store it in memory.

#### Naive Approach

Let us carry on with the example of Google search queries, we might imagine that Google receives the following tuples in a stream `(user, query, time)`. If we could get a 10% sample of the data stream we can estimate the answer to the question "How often did a user query the same at least twice?".

Our first naive approach might be that we have a 10-sided dice and every time we receive a new tuple we roll the die and if it hits 1 we add the tuple to the sample. If we would be programming this instead of throwing a dice we could generate a random number between 1 and 10, $r \in [1,10]$ and if $r=1$ we store the sample.

#### Bucket Approach

### Fixed Size Sample

## Queries Over a Sliding Window

### DGIM Method

## Filtering a Data Stream

### Bloom Filter

## Counting Distinct Elements

### Flajolet–Martin algorithm

## Moments

### AMS Method

## Counting Frequent Itemsets